In [1]:
# default_exp models.nbeats.onnbeats

In [2]:
#hide
%load_ext autoreload
%autoreload 2

In [3]:
#export
import os
import time
import numpy as np
import pandas as pd
import random
from collections import defaultdict
import copy

import torch as t
from torch import optim
from pathlib import Path
from functools import partial

from nixtla.models.nbeats.onnbeats_model import NBeats, NBeatsBlock, IdentityBasis
from nixtla.models.nbeats.onnbeats_model import TrendBasis, SeasonalityBasis, ExogenousFutureBasis, ExogenousBasisInterpretable
from nixtla.losses.pytorch import MAPELoss, MASELoss, SMAPELoss, MSELoss, MAELoss
from nixtla.losses.numpy import mae, mse, mape, smape, rmse

In [4]:
#export
class Nbeats(object):
    """
    Future documentation
    """
    SEASONALITY_BLOCK = 'seasonality'
    TREND_BLOCK = 'trend'
    IDENTITY_BLOCK = 'identity'

    def __init__(self,
                 input_size_multiplier,
                 output_size,
                 shared_weights,
                 stack_types,
                 n_blocks,
                 n_layers,
                 n_hidden,
                 n_harmonics,
                 n_polynomials,
                 exogenous_n_channels,
                 f_cols,
                 theta_with_exogenous,
                 batch_normalization,
                 dropout_prob,
                 x_s_n_hidden,
                 learning_rate,
                 lr_decay,
                 n_lr_decay_steps,
                 l1_lambda_x,
                 weight_decay,
                 n_iterations,
                 early_stopping,
                 loss,
                 frequency,
                 seasonality,
                 random_seed,
                 device=None):
        super(Nbeats, self).__init__()

        self.input_size = int(input_size_multiplier*output_size)
        self.output_size = output_size
        self.shared_weights = shared_weights
        self.stack_types = stack_types
        self.n_blocks = n_blocks
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.n_harmonics = n_harmonics
        self.n_polynomials = n_polynomials
        self.exogenous_n_channels = exogenous_n_channels
        self.f_cols = f_cols
        self.theta_with_exogenous = theta_with_exogenous
        self.batch_normalization = batch_normalization
        self.dropout_prob = dropout_prob
        self.x_s_n_hidden = x_s_n_hidden
        self.learning_rate = learning_rate
        self.lr_decay = lr_decay
        self.n_lr_decay_steps = n_lr_decay_steps
        self.l1_lambda_x = l1_lambda_x
        self.weight_decay = weight_decay
        self.n_iterations = n_iterations
        self.early_stopping = early_stopping
        self.loss = loss
        self.frequency = frequency
        self.seasonality = seasonality
        self.random_seed = random_seed
        if device is None:
            device = 'cuda' if t.cuda.is_available() else 'cpu'
        self.device = device

        self._is_instantiated = False

    def create_stack(self):
        # Declare parameter dimensions
        if self.theta_with_exogenous:
            x_t_n_inputs = self.input_size + self.n_x_t
        else:
            x_t_n_inputs = self.input_size # y_lags
        
        #------------------------ Model Definition ------------------------#
        block_list = []
        self.blocks_regularizer = []
        for i in range(len(self.stack_types)):
            #print(f'| --  Stack {self.stack_types[i]} (#{i})')
            for block_id in range(self.n_blocks[i]):
                # Batch norm only on first block
                if (len(block_list)==0) and (self.batch_normalization):
                    batch_normalization_block = True
                else:
                    batch_normalization_block = False

                # Dummy of regularizer in block. Override with 1 if exogenous_block
                #self.blocks_regularizer += [0]

                # Shared weights
                if self.shared_weights and block_id>0:
                    nbeats_block = block_list[-1]

                else:
                    if self.stack_types[i] == 'seasonality':
                        nbeats_block = NBeatsBlock(x_t_n_inputs = x_t_n_inputs,
                                                   x_s_n_inputs = self.n_x_s,
                                                   x_s_n_hidden= self.x_s_n_hidden,
                                                   theta_n_dim=4 * int(
                                                        np.ceil(self.n_harmonics / 2 * self.output_size) - (self.n_harmonics - 1)),
                                                   basis=SeasonalityBasis(harmonics=self.n_harmonics,
                                                                                backcast_size=self.input_size,
                                                                                forecast_size=self.output_size),
                                                   n_layers=self.n_layers[i],
                                                   theta_n_hidden=self.n_hidden[i],
                                                   theta_with_exogenous=self.theta_with_exogenous,
                                                   batch_normalization=batch_normalization_block,
                                                   dropout_prob=self.dropout_prob)
                    elif self.stack_types[i] == 'trend':
                        nbeats_block = NBeatsBlock(x_t_n_inputs = x_t_n_inputs,
                                                   x_s_n_inputs = self.n_x_s,
                                                   x_s_n_hidden= self.x_s_n_hidden,
                                                   theta_n_dim=2 * (self.n_polynomials + 1),
                                                   basis=TrendBasis(degree_of_polynomial=self.n_polynomials,
                                                                            backcast_size=self.input_size,
                                                                            forecast_size=self.output_size),
                                                   n_layers=self.n_layers[i],
                                                   theta_n_hidden=self.n_hidden[i],
                                                   theta_with_exogenous=self.theta_with_exogenous,
                                                   batch_normalization=batch_normalization_block,
                                                   dropout_prob=self.dropout_prob)
                    elif self.stack_types[i] == 'identity':
                        nbeats_block = NBeatsBlock(x_t_n_inputs = x_t_n_inputs,
                                                   x_s_n_inputs = self.n_x_s,
                                                   x_s_n_hidden= self.x_s_n_hidden,
                                                   theta_n_dim=self.input_size + self.output_size,
                                                   basis=IdentityBasis(backcast_size=self.input_size,
                                                                       forecast_size=self.output_size),
                                                   n_layers=self.n_layers[i],
                                                   theta_n_hidden=self.n_hidden[i],
                                                   theta_with_exogenous=self.theta_with_exogenous,
                                                   batch_normalization=batch_normalization_block,
                                                   dropout_prob=self.dropout_prob)
                    elif self.stack_types[i] == 'exogenous':
                        nbeats_block = NBeatsBlock(x_t_n_inputs = x_t_n_inputs,
                                                   x_s_n_inputs = self.n_x_s,
                                                   x_s_n_hidden= self.x_s_n_hidden,
                                                   theta_n_dim=2*self.n_x_t,
                                                   basis=ExogenousBasisInterpretable(),
                                                   n_layers=self.n_layers[i],
                                                   theta_n_hidden=self.n_hidden[i],
                                                   theta_with_exogenous=self.theta_with_exogenous,
                                                   batch_normalization=batch_normalization_block,
                                                   dropout_prob=self.dropout_prob)
                    elif self.stack_types[i] == 'exogenous_g_a':
                        assert len(self.f_cols)>0, 'If ExogenousFutureBasis, provide x_f_cols hyperparameter'
                        nbeats_block = NBeatsBlock(x_t_n_inputs = x_t_n_inputs,
                                                   x_s_n_inputs = self.n_x_s,
                                                   x_s_n_hidden= self.x_s_n_hidden,
                                                   theta_n_dim=2*(self.exogenous_n_channels),
                                                   basis=ExogenousFutureBasis(out_features=self.exogenous_n_channels,
                                                                              f_idxs=self.f_idxs),
                                                   n_layers=self.n_layers[i],
                                                   theta_n_hidden=self.n_hidden[i],
                                                   theta_with_exogenous=self.theta_with_exogenous,
                                                   batch_normalization=batch_normalization_block,
                                                   dropout_prob=self.dropout_prob)
                #        self.blocks_regularizer[-1] = 1
                #print(f'     | -- {nbeats_block}')
                block_list.append(nbeats_block)
        return block_list

    def __loss_fn(self, loss_name: str):
        def loss(x, freq, forecast, target, mask):
            if loss_name == 'MAPE':
                return MAPELoss(y=target, y_hat=forecast, mask=mask) + self.l1_regularization()
            elif loss_name == 'MASE':
                return MASELoss(y=target, y_hat=forecast, y_insample=x, seasonality=freq, mask=mask) + self.l1_regularization()
            elif loss_name == 'SMAPE':
                return SMAPELoss(y=target, y_hat=forecast, mask=mask) + self.l1_regularization()
            elif loss_name == 'MSE':
                return MSELoss(y=target, y_hat=forecast, mask=mask) + self.l1_regularization()
            elif loss_name == 'RMSE':
                return RMSELoss(y=target, y_hat=forecast, mask=mask) + self.l1_regularization()
            elif loss_name == 'MAE':
                return MAELoss(y=target, y_hat=forecast, mask=mask) + self.l1_regularization()
            else:
                raise Exception(f'Unknown loss function: {loss_name}')
        return loss

    def __val_loss_fn(self, loss_name: str):
        #TODO: mase not implemented
        def loss(forecast, target, weights):
            if loss_name == 'MAPE':
                return mape(y=target, y_hat=forecast, weights=weights) #TODO: faltan weights
            elif loss_name == 'SMAPE':
                return smape(y=target, y_hat=forecast, weights=weights) #TODO: faltan weights
            elif loss_name == 'MSE':
                return mse(y=target, y_hat=forecast, weights=weights)
            elif loss_name == 'RMSE':
                return rmse(y=target, y_hat=forecast, weights=weights)
            elif loss_name == 'MAE':
                return mae(y=target, y_hat=forecast, weights=weights)
            else:
                raise Exception(f'Unknown loss function: {loss_name}')
        return loss

    def l1_regularization(self):
        # l1_loss = 0
        # for i, indicator in enumerate(self.blocks_regularizer):
        #     if indicator:
        #         l1_loss +=  self.l1_lambda*t.sum(t.abs(self.model.blocks[i].basis.weight))
        # return l1_loss
        return self.l1_lambda_x * t.sum(t.abs(self.model.l1_weight))

    def to_tensor(self, x: np.ndarray) -> t.Tensor:
        tensor = t.as_tensor(x, dtype=t.float32).to(self.device)
        return tensor

    def evaluate_performance(self, ts_loader, validation_loss_fn):
        #TODO: mas opciones que mae
        self.model.eval()

        losses = []
        with t.no_grad():
            for batch in iter(ts_loader):
                insample_y     = self.to_tensor(batch['insample_y'])
                insample_x     = self.to_tensor(batch['insample_x'])
                insample_mask  = self.to_tensor(batch['insample_mask'])
                outsample_x    = self.to_tensor(batch['outsample_x'])
                outsample_y    = self.to_tensor(batch['outsample_y'])
                outsample_mask = self.to_tensor(batch['outsample_mask'])
                s_matrix       = self.to_tensor(batch['s_matrix'])

                forecast = self.model(insample_y=insample_y, insample_x_t=insample_x,
                                    insample_mask=insample_mask, outsample_x_t=outsample_x, x_s=s_matrix)
                batch_loss = validation_loss_fn(target=forecast.cpu().data.numpy(),
                                                forecast=outsample_y.cpu().data.numpy(),
                                                weights=outsample_mask.cpu().data.numpy())
                losses.append(batch_loss)
                break #TODO: remove this in future
        loss = np.mean(losses)
        self.model.train()
        return loss

    def fit(self, train_ts_loader, val_ts_loader=None, n_iterations=None, verbose=True, eval_steps=1):
        # Asserts
        assert train_ts_loader.t_cols[0] == 'y', f'First variable must be y not {train_ts_loader.t_cols[0]}'
        assert train_ts_loader.t_cols[1] == 'ejecutado', f'First exogenous variable must be ejecutado not {train_ts_loader.t_cols[1]}'
        assert (self.input_size)==train_ts_loader.input_size, \
            f'model input_size {self.input_size} data input_size {train_ts_loader.input_size}'        

        # Random Seeds (model initialization)
        t.manual_seed(self.random_seed)
        np.random.seed(self.random_seed)
        random.seed(self.random_seed) #TODO: interaccion rara con window_sampling de validacion

        # Attributes of ts_dataset
        self.n_x_t, self.n_x_s = train_ts_loader.get_n_variables()
        self.t_cols = train_ts_loader.t_cols
        self.f_idxs = train_ts_loader.ts_dataset.get_f_idxs(self.f_cols)

        # Instantiate model
        if not self._is_instantiated:
            block_list = self.create_stack()
            self.model = NBeats(blocks=t.nn.ModuleList(block_list), in_features=self.n_x_t).to(self.device)
            self._is_instantiated = True

        # Overwrite n_iterations and train datasets
        if n_iterations is None:
            n_iterations = self.n_iterations

        train_dataloader = iter(train_ts_loader)

        lr_decay_steps = n_iterations // self.n_lr_decay_steps
        if lr_decay_steps == 0:
            lr_decay_steps = 1

        optimizer = optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay)
        lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=lr_decay_steps, gamma=self.lr_decay)

        training_loss_fn = self.__loss_fn(self.loss)
        validation_loss_fn = self.__val_loss_fn(self.loss) #Uses numpy losses

        if verbose and (n_iterations > 0):
            print('='*30+' Start fitting '+'='*30)
            print(f'Number of exogenous variables: {self.n_x_t}')
            print(f'Number of static variables: {self.n_x_s} , with dim_hidden: {self.x_s_n_hidden}')
            print(f'Number of iterations: {n_iterations}')
            print(f'Number of blocks: {len(self.model.blocks)}')

        #self.loss_dict = {} # Restart self.loss_dict
        start = time.time()
        self.trajectories = {'step':[],'train_loss':[], 'val_loss':[]}
        self.final_insample_loss = None
        self.final_outsample_loss = None

        # Training Loop
        best_val_loss = np.inf
        break_flag = False
        for step in range(n_iterations):
            self.model.train()
            train_ts_loader.train()

            batch = next(train_dataloader)
            insample_y     = self.to_tensor(batch['insample_y'])
            insample_x     = self.to_tensor(batch['insample_x'])
            insample_mask  = self.to_tensor(batch['insample_mask'])
            outsample_x    = self.to_tensor(batch['outsample_x'])
            outsample_y    = self.to_tensor(batch['outsample_y'])
            outsample_mask = self.to_tensor(batch['outsample_mask'])
            s_matrix       = self.to_tensor(batch['s_matrix'])

            optimizer.zero_grad()
            forecast = self.model(insample_y=insample_y, insample_x_t=insample_x,
                                  insample_mask=insample_mask, outsample_x_t=outsample_x, x_s=s_matrix)

            training_loss = training_loss_fn(x=insample_y, freq=self.seasonality, forecast=forecast,
                                            target=outsample_y, mask=outsample_mask)

            if np.isnan(float(training_loss)):
                break

            training_loss.backward()
            t.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            optimizer.step()

            lr_scheduler.step()
            if (step % eval_steps == 0):
                display_string = 'Step: {}, Time: {:03.3f}, Insample {}: {:.5f}'.format(step,
                                                                                time.time()-start,
                                                                                self.loss,
                                                                                training_loss.cpu().data.numpy())
                self.trajectories['step'].append(step)
                self.trajectories['train_loss'].append(training_loss.cpu().data.numpy())

                if val_ts_loader is not None:
                    loss = self.evaluate_performance(val_ts_loader, validation_loss_fn=validation_loss_fn)
                    display_string += ", Outsample {}: {:.5f}".format(self.loss, loss)
                    self.trajectories['val_loss'].append(loss)

                    if self.early_stopping:
                        if loss < best_val_loss:
                            # Save current model if improves outsample loss
                            best_state_dict = copy.deepcopy(self.model.state_dict())
                            best_insample_loss = training_loss.cpu().data.numpy()
                            early_stopping_counter = 0
                            best_val_loss = loss
                        else:
                            early_stopping_counter += 1
                        if early_stopping_counter >= self.early_stopping:
                            break_flag = True
                
                print(display_string)

                self.model.train()
                train_ts_loader.train()

            if break_flag:
                print(10*'-',' Stopped training by early stopping', 10*'-')
                self.model.load_state_dict(best_state_dict)
                break

        #End of fitting
        if n_iterations >0:
            self.final_insample_loss = training_loss.cpu().data.numpy() if not break_flag else best_insample_loss #This is batch!
            string = 'Step: {}, Time: {:03.3f}, Insample {}: {:.5f}'.format(step,
                                                                            time.time()-start,
                                                                            self.loss,
                                                                            self.final_insample_loss)
            if val_ts_loader is not None:
                self.final_outsample_loss = self.evaluate_performance(val_ts_loader, validation_loss_fn=validation_loss_fn)
                string += ", Outsample {}: {:.5f}".format(self.loss, self.final_outsample_loss)
            print(string)
            print('='*30+'End fitting '+'='*30)

    def predict(self, ts_loader, X_test=None, eval_mode=False):

        ts_loader.eval()
        frequency = ts_loader.get_frequency()

        # Build forecasts
        unique_ids = ts_loader.get_meta_data_var('unique_id')
        last_ds = ts_loader.get_meta_data_var('last_ds') #TODO: ajustar of offset

        batch = next(iter(ts_loader))
        insample_y     = self.to_tensor(batch['insample_y'])
        insample_x     = self.to_tensor(batch['insample_x'])
        insample_mask  = self.to_tensor(batch['insample_mask'])
        outsample_x    = self.to_tensor(batch['outsample_x'])
        outsample_y    = self.to_tensor(batch['outsample_y'])
        outsample_mask = self.to_tensor(batch['outsample_mask'])
        s_matrix       = self.to_tensor(batch['s_matrix'])

        self.model.eval()
        with t.no_grad():
            forecast = self.model(insample_y=insample_y, insample_x_t=insample_x,
                                  insample_mask=insample_mask, outsample_x_t=outsample_x, x_s=s_matrix)

        if eval_mode:
            return forecast, outsample_y, outsample_mask

        # Predictions for panel
        Y_hat_panel = pd.DataFrame(columns=['unique_id', 'ds'])
        for i, unique_id in enumerate(unique_ids):
            Y_hat_id = pd.DataFrame([unique_id]*self.output_size, columns=["unique_id"])
            ds = pd.date_range(start=last_ds[i], periods=self.output_size+1, freq=self.frequency)
            Y_hat_id["ds"] = ds[1:]
            Y_hat_panel = Y_hat_panel.append(Y_hat_id, sort=False).reset_index(drop=True)

        forecast = forecast.cpu().detach().numpy()
        Y_hat_panel['y_hat'] = forecast.flatten()

        if X_test is not None:
            Y_hat_panel = X_test.merge(Y_hat_panel, on=['unique_id', 'ds'], how='left')

        return Y_hat_panel


    def save(self, model_dir, model_id):

        if not os.path.exists(model_dir):
            os.makedirs(model_dir)

        model_file = os.path.join(model_dir, f"model_{model_id}.model")
        print('Saving model to:\n {}'.format(model_file)+'\n')
        t.save({'model_state_dict': self.model.state_dict()}, model_file)

    def load(self, model_dir, model_id):

        model_file = os.path.join(model_dir, f"model_{model_id}.model")
        path = Path(model_file)

        assert path.is_file(), 'No model_*.model file found in this path!'

        print('Loading model from:\n {}'.format(model_file)+'\n')

        checkpoint = t.load(model_file, map_location=self.device)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.model.to(self.device)

In [5]:
import time
import numpy as np
import pandas as pd
import random
import torch as t
import copy
from fastcore.foundation import patch
from nixtla.data.ontsdataset import TimeSeriesDataset
from nixtla.data.ontsloader_fast import TimeSeriesLoader as TimeSeriesLoaderFast
# from nixtla.data.tsloader_pinche import TimeSeriesLoader as TimeSeriesLoaderPinche
# from nixtla.data.tsloader_general import TimeSeriesLoader as TimeSeriesLoaderGeneral

#from nixtla.models.nbeats.onnbeats import Nbeats
from nixtla.data.datasets.on import load_on_data
np.random.seed(1)
t.manual_seed(1)

Y_insample_df, X_insample_df, Y_outsample_df, X_outsample_df, f_cols = load_on_data('2020-09-01')

ts_train_mask = np.ones(len(Y_insample_df))
ts_train_mask[-7*6*16:] = 0 # 16 fifteenminutales = 4 hours   (total = 1 week)
dataset = TimeSeriesDataset(Y_df=Y_insample_df, S_df=None, X_df=X_insample_df, ts_train_mask=ts_train_mask, f_cols=f_cols)
print('X: time series features, of shape (#series,#times,#features): \t' + str(X_insample_df.shape))
print('Y: target series (in X), of shape (#series,#times): \t \t' + str(Y_insample_df.shape))
# print('S: static features, of shape (#series,#features): \t \t' + str(S.shape))
Y_insample_df.head()

Processing dataframes ...
Creating ts tensor ...
X: time series features, of shape (#series,#times,#features): 	(101760, 331)
Y: target series (in X), of shape (#series,#times): 	 	(101760, 3)


,unique_id,ds,y
0,demanda_final,2017-10-07 00:00:00,5545.98338
1,demanda_final,2017-10-07 00:15:00,5448.66753
2,demanda_final,2017-10-07 00:30:00,5392.92989
3,demanda_final,2017-10-07 00:45:00,5371.67098
4,demanda_final,2017-10-07 01:00:00,5296.58436


In [6]:
Y_insample_df.tail()

,unique_id,ds,y
101755,demanda_final,2020-08-31 22:45:00,5911.97322
101756,demanda_final,2020-08-31 23:00:00,5819.01473
101757,demanda_final,2020-08-31 23:15:00,5750.59348
101758,demanda_final,2020-08-31 23:30:00,5639.96684
101759,demanda_final,2020-08-31 23:45:00,5589.45439


In [7]:
train_loader = TimeSeriesLoaderFast(ts_dataset=dataset,
                                    model='nbeats',
                                    offset=0,
                                    window_sampling_limit=60*6*16, 
                                    input_size=3*16,
                                    output_size=16,
                                    idx_to_sample_freq=1,
                                    batch_size=256,
                                    is_train_loader=True)

val_loader = TimeSeriesLoaderFast(ts_dataset=dataset,
                                  model='nbeats',
                                  offset=0,
                                  window_sampling_limit=60*6*16,
                                  input_size=3*16,
                                  output_size=16,
                                  idx_to_sample_freq=1,
                                  batch_size=256,
                                  is_train_loader=False)

In [8]:
start = time.time()
dataloader = iter(train_loader)
batch = next(dataloader)
insample_y = batch['insample_y']
insample_x = batch['insample_x']
insample_mask = batch['insample_mask']
outsample_x = batch['outsample_x']
outsample_y = batch['outsample_y']
outsample_mask = batch['outsample_mask']
print("DataloaderGeneral batch time:", time.time()-start)
print("insample_y.shape", insample_y.shape)
print("insample_x.shape", insample_x.shape)
print("outsample_y.shape", outsample_y.shape)
print("outsample_x.shape", outsample_x.shape)
print("t.max(insample_y)", t.max(insample_y))
print("t.max(outsample_y)", t.max(outsample_y * outsample_mask))
insample_y

DataloaderGeneral batch time: 0.021054983139038086
insample_y.shape torch.Size([256, 48])
insample_x.shape torch.Size([256, 328, 48])
outsample_y.shape torch.Size([256, 16])
outsample_x.shape torch.Size([256, 328, 16])
t.max(insample_y) tensor(6632.9390)
t.max(outsample_y) tensor(6552.0488)


tensor([[6049.8535, 6049.8535, 6028.8916,  ..., 5271.8174, 5211.4722,
         5211.4722],
        [5360.6880, 5370.7568, 5370.7568,  ..., 4879.2607, 4879.2607,
         4897.5063],
        [   0.0000,    0.0000,    0.0000,  ...,    0.0000,    0.0000,
         5195.6118],
        ...,
        [5044.3916, 5044.3916, 4904.4316,  ..., 5408.3057, 5447.9019,
         5447.9019],
        [6160.2803, 6084.2627, 6084.2627,  ..., 5385.7246, 5385.7246,
         5359.4512],
        [6096.0269, 6093.2905, 6093.2905,  ..., 6293.4448, 6293.4448,
         6130.4795]])

In [9]:
nbeatsx = Nbeats(input_size_multiplier=3,
                 output_size=16,
                 shared_weights=False,
                 stack_types=['exogenous_g_a']+3*['identity'],
                 n_blocks=4*[1],
                 n_layers=4*[2],
                 n_hidden=4*[256],
                 n_harmonics=1,
                 n_polynomials=2,
                 x_s_n_hidden=0,
                 exogenous_n_channels=9,
                 f_cols=f_cols,
                 batch_normalization=False,
                 dropout_prob=0.1,
                 theta_with_exogenous=True,
                 learning_rate=0.001,
                 lr_decay=0.5,
                 n_lr_decay_steps=3,
                 weight_decay=0.0000001,
                 l1_lambda_x=0.0001,
                 n_iterations=100,
                 early_stopping=10,
                 loss='MAE',
                 frequency=24,
                 random_seed=1,
                 seasonality='H')

nbeatsx.fit(train_ts_loader=train_loader, val_ts_loader=val_loader, verbose=True, eval_steps=5)

============================== Start fitting ==============================
Number of exogenous variables: 328
Number of static variables: 0 , with dim_hidden: 0
Number of iterations: 100
Number of blocks: 4
Step: 0, Time: 0.549, Insample MAE: 353.87540, Outsample MAE: 611.06189
Step: 5, Time: 3.413, Insample MAE: 320.83377, Outsample MAE: 296.92816
Step: 10, Time: 6.292, Insample MAE: 265.39258, Outsample MAE: 263.11163
Step: 15, Time: 9.420, Insample MAE: 290.49939, Outsample MAE: 290.93185
Step: 20, Time: 12.327, Insample MAE: 316.88730, Outsample MAE: 243.43793
Step: 25, Time: 15.079, Insample MAE: 284.27936, Outsample MAE: 326.36108
Step: 30, Time: 18.057, Insample MAE: 260.05319, Outsample MAE: 260.20950
Step: 35, Time: 20.740, Insample MAE: 244.63890, Outsample MAE: 244.00592
Step: 40, Time: 23.423, Insample MAE: 233.15315, Outsample MAE: 255.98195
Step: 45, Time: 26.113, Insample MAE: 266.68124, Outsample MAE: 229.89787
Step: 50, Time: 28.796, Insample MAE: 262.22791, Outsample